# Experiments in reproducing the LSTM architecture for robot control using Torch.

In [ ]:
import sys
sys.path.append("..")

from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

from gen_synthetic_data import generate_training_sequence, create_training_sequence_prediction, create_training_sequence_xy

## Training data

Generating predictable and unpredictable training data. 

In [ ]:
# Experiments in visualizing the generated training sequence
total_length = 100
# latent_size = 6
x_seq, y_seq = generate_training_sequence(total_length=100, latent_size=7)
plt.plot(y_seq[0:100,3], label= "3")
plt.plot(x_seq[0:100,4], label= "4")
plt.plot(x_seq[0:100,5], label= "5")
plt.legend()

In [ ]:

sequence_length = 10  # Number of vectors in input sequence
inputs, targets = create_training_sequence_prediction(x_seq, sequence_length=sequence_length)
# Verify shapes
print("Inputs shape:", inputs.shape)   # Expected: [num_samples, sequence_length, latent_size]
print("Targets shape:", targets.shape) # Expected: [num_samples, latent_size]

## LSTM model for sequence prediction
Create an LSTM model for predicting a sequence. Train it. 

In [ ]:
# This is the architecture created by chatgpt
class LSTMSequencePredictor(nn.Module):
    def __init__(self, latent_size, hidden_size, num_layers):
        super(LSTMSequencePredictor, self).__init__()
        self.lstm = nn.LSTM(latent_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, latent_size)

    def forward(self, x):
        # x: [batch_size, sequence_length, latent_size]
        out, _ = self.lstm(x)  # LSTM output shape: [batch_size, sequence_length, hidden_size]
        out = self.fc(out[:, -1, :])  # Take last time step output and pass through the fully connected layer
        return out  # Predicted next vector

In [ ]:
# LSTM controller with residuals and three layers
# This code is written by Lotzi
class LSTMResidualController(nn.Module):
    def __init__(self, latent_size, hidden_size, output_size):
        super(LSTMResidualController, self).__init__()
        self.lstm_1 = nn.LSTM(latent_size, hidden_size, num_layers=1, batch_first=True)

        self.lstm_2 = nn.LSTM(hidden_size, hidden_size, num_layers=1, batch_first=True)

        self.lstm_3 = nn.LSTM(hidden_size, hidden_size, num_layers=1, batch_first=True)

        self.fc = nn.Linear(hidden_size, latent_size)

    def forward(self, x):
        # x: [batch_size, sequence_length, latent_size]
        out_1, _ = self.lstm_1(x)
        residual = out_1
        out_2, _ = self.lstm_2(out_1)
        out_2 = out_2 + residual
        residual = out_2
        out_3, _ = self.lstm_3(out_2)
        out_3 = out_3 + residual

        # LSTM output shape: [batch_size, sequence_length, hidden_size]
        out = self.fc(out_3[:, -1, :])  # Take last time step output and pass through the fully connected layer
        return out  # Predicted next vector

In [ ]:

# Model Parameters
# Original
# latent_size = 7  # Dimension of each vector in the sequence
# hidden_size = 32  # Size of hidden state in LSTM
# num_layers = 2    # Number of LSTM layers

# Original
latent_size = 7  # Dimension of each vector in the sequence
hidden_size = 3  # Size of hidden state in LSTM
num_layers = 1    # Number of LSTM layers

# Simple model: Instantiate model, loss function, and optimizer
model = LSTMSequencePredictor(latent_size=latent_size, hidden_size=hidden_size, num_layers=num_layers)

# Residual model: Instantiate model, loss function, and optimizer
# model = LSTMResidualController(latent_size=latent_size, hidden_size=hidden_size, output_size = hidden_size)

criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create the training data
total_length = 100
x_seq, y_seq = generate_training_sequence(total_length=total_length, latent_size=latent_size)
inputs, targets = create_training_sequence_prediction(x_seq, sequence_length=sequence_length)
num_sequences = inputs.shape[0]
print(num_sequences)

In [ ]:

# Training Loop
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    
    # Loop over each sequence in the batch
    for i in range(num_sequences):
        # Prepare input and target
        input_seq = inputs[i]
        target = targets[i]

        # Reshape for batch compatibility
        input_seq = input_seq.unsqueeze(0)  # Shape: [1, sequence_length, latent_size]
        target = target.unsqueeze(0)        # Shape: [1, latent_size]

        # Forward pass
        output = model(input_seq)
        loss = criterion(output, target)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 2 == 0: # was 0
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete.")

# FIXME: save the model
# FIXME: this should be saved into some experimental file name. 
filename_lstm = Config()["explorations"]["lstm_model_file"]
torch.save(model.state_dict(), filename_lstm)

In [ ]:
load_model = True
if load_model:
    model = LSTMSequencePredictor(latent_size=latent_size, hidden_size=hidden_size, num_layers=num_layers)
    filename_lstm = Config()["explorations"]["lstm_model_file"]
    model.load_state_dict(torch.load(filename_lstm))
# FIXME: load the model

## Evaluate the trained LSTM model

In [ ]:
def model_step_by_step_prediction(model, sequence_length, latent_size):
    """Run a prediction where we are always predicting the next item from the previous 10 ground truth items"""
    start = 0
    result = np.zeros((sequence_length, latent_size))
    model.eval()
    with torch.no_grad():
        for i in range(0, total_length - sequence_length):
            input_seq = x_seq[start+i : start+ i + sequence_length]
            input_val = input_seq.unsqueeze(0)
            val = model(input_val)
            result = np.append(result, [val.squeeze(0).tolist()], axis=0) 
    return result

In [ ]:


# Visualize the step by step prediction
result = model_step_by_step_prediction(model, sequence_length, latent_size)
print(result.shape)

channel = 2
plt.plot(result[0:100,channel], label=f"result {channel}")
plt.plot(x_seq[0:100,channel], label = f"x_seq {channel}")
plt.legend()


In [ ]:
# Run autoregressive prediction (use the prediction as the next data)
def model_autoregressive_prediction(model, x_seq, sequence_length):
    """Run an autoregressive prediction where we are always predicting the next item from the previous 10 ground truth items"""
    start = 0
    result = np.zeros((sequence_length, latent_size))
    input_seq = x_seq[start : start + sequence_length]
    with torch.no_grad():
        for i in range(0, total_length - sequence_length):
            model.eval()            
            input_val = input_seq.unsqueeze(0)
            val = model(input_val)
            next_item = val.squeeze(0).tolist()
            result = np.append(result, [val.squeeze(0).tolist()], axis=0) 
            input_seq = input_seq[1: sequence_length]
            input_seq= torch.cat((input_seq, val))
    #print(result)
    return np.array(result)

In [ ]:
# Visualize the autoregressive prediction
# Calculate the error of the autoregressive prediction
# Visualize the step by step prediction
result = model_autoregressive_prediction(model, x_seq, sequence_length)
channel = 4
plt.plot(result[1:100,channel])
plt.plot(x_seq[1:100,channel])

## Training with an output which is not prediction

Training an LSTM with an output that is not a prediction of a string, but has a different dimensionality. 

In [ ]:
# This is the architecture created by chatgpt
class LSTMXYPredictor(nn.Module):
    def __init__(self, latent_size, hidden_size, output_size, num_layers):
        super(LSTMXYPredictor, self).__init__()
        self.lstm = nn.LSTM(latent_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x: [batch_size, sequence_length, latent_size]
        out, _ = self.lstm(x)  # LSTM output shape: [batch_size, sequence_length, hidden_size]
        out = self.fc(out[:, -1, :])  # Take last time step output and pass through the fully connected layer
        return out  # Predicted next vector

In [ ]:
# Model Parameters
# Original
latent_size = 7  # Dimension of each vector in the sequence
hidden_size = 32  # Size of hidden state in LSTM
num_layers = 2    # Number of LSTM layers

# Modified
# latent_size = 7  # Dimension of each vector in the sequence
# hidden_size = 3  # Size of hidden state in LSTM
# num_layers = 1    # Number of LSTM layers
output_size = 5

# Instantiate model, loss function, and optimizer
model = LSTMXYPredictor(latent_size=latent_size, hidden_size=hidden_size, output_size = output_size, num_layers=num_layers)
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create the training data
total_length = 100
x_seq, y_seq = generate_training_sequence(total_length=total_length, latent_size=latent_size, y_size=output_size)
inputs, targets = create_training_sequence_xy(x_seq, y_seq, sequence_length=sequence_length)
num_sequences = inputs.shape[0]
print(inputs.shape)
print(targets.shape)

In [ ]:
# Training Loop
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    
    # Loop over each sequence in the batch
    for i in range(num_sequences):
        # Prepare input and target
        input_seq = inputs[i]
        target = targets[i]

        # Reshape for batch compatibility
        input_seq = input_seq.unsqueeze(0)  # Shape: [1, sequence_length, latent_size]
        target = target.unsqueeze(0)        # Shape: [1, latent_size]

        # Forward pass
        output = model(input_seq)
        loss = criterion(output, target)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 2 == 0: # was 0
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete.")

# FIXME: save the model
filename_lstm = Config()["controller"]["lstm_model_file"]
torch.save(model.state_dict(), filename_lstm+".2")

In [ ]:
# FIXME: not implemented yet

def model_xy_prediction(model, x_seq, sequence_length, latent_size, output_size):
    """Model the XY prediction"""
    start = 0
    result = np.zeros((sequence_length, output_size))
    model.eval()
    with torch.no_grad():
        for i in range(0, total_length - sequence_length):
            input_seq = x_seq[start+i : start+ i + sequence_length]
            input_val = input_seq.unsqueeze(0)
            val = model(input_val)
            result = np.append(result, [val.squeeze(0).tolist()], axis=0) 
    return result

In [ ]:
# Visualize the step by step prediction
result = model_xy_prediction(model, x_seq, sequence_length, latent_size, output_size)
print(result.shape)

channel = 4
plt.plot(result[0:100,channel], label=f"result {channel}")
plt.plot(y_seq[0:100,channel], label = f"y_seq {channel}")
plt.legend()
